### Install pyfor

1. Create a conda environment for this project. You should have installed conda or miniconda already for this to run.
```
conda env create -f ./3dworkbench.yml
```
2. Install pyfor library and dependancies. This will take a while due to dependancies heavy files.

3. If necessary, convert the LAZ file to LAS (You can do this using the LASTools Plugin on QGIS)

## Import Libraries

In [1]:
import laspy
import open3d as o3d
import numpy as np
import trimesh as tm
import shapely
import geopandas

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Import Files

In [2]:
las = laspy.read('../Data/OldCityEnschede.las')

### Transform data into a Open3D point cloud and visualize

In [3]:
point_data = np.stack([las.X, las.Y, las.Z], axis=0).transpose((1,0))

In [4]:
geom = o3d.geometry.PointCloud()
geom.points = o3d.utility.Vector3dVector(point_data)


In [5]:
o3d.visualization.draw_geometries([geom])

In [14]:
## convert open3D object into a np array and transform into meters.
xyz_load = (np.asarray(geom.points))/1000
xyz_load

array([[2.57999774e+05, 4.71120324e+05, 4.06530000e+01],
       [2.57999241e+05, 4.71121535e+05, 4.05820000e+01],
       [2.57999032e+05, 4.71122013e+05, 4.06360000e+01],
       ...,
       [2.58106560e+05, 4.71250658e+05, 5.16350000e+01],
       [2.58100464e+05, 4.71251357e+05, 4.20990000e+01],
       [2.58104199e+05, 4.71250414e+05, 4.23260000e+01]])

In [16]:
#get boundaries and size of the diagonal

minb= np.min(xyz_load, axis=0)
maxb= np.max(xyz_load, axis=0)

diagonal = (maxb - minb)
diagonal

array([375.69 , 413.681,  54.52 ])

In [17]:
bbox = (minb,maxb)
bbox

(array([2.57918444e+05, 4.71075511e+05, 3.99730000e+01]),
 array([2.58294134e+05, 4.71489192e+05, 9.44930000e+01]))

In [28]:
pixsize =np.array([10,10])

bboxR2 = ([minb[0],minb[1]],[maxb[0],maxb[1]])
bboxR2

([257918.444, 471075.511], [258294.134, 471489.192])

In [31]:
bboxN2 = bboxR2/pixsize
bboxN2

array([[25791.8444, 47107.5511],
       [25829.4134, 47148.9192]])

In [33]:
bboxN2=np.rint(bboxN2).astype(int)
bboxN2

array([[25792, 47108],
       [25829, 47149]])

In [36]:
diagonalN2=bboxN2[1]-bboxN2[0]
diagonalN2 

(m,n)=diagonalN2
m,n

(37, 41)

In [38]:
pixelsR2=[]
for i in range(m):
    for j in range(n):
        pixelN2 = [i,j]
        pixelR2 = [(pixelN2*pixsize)+bboxR2[0]]
        pixelsR2.append(pixelR2)

pixelsR2



[[array([257918.444, 471075.511])],
 [array([257918.444, 471085.511])],
 [array([257918.444, 471095.511])],
 [array([257918.444, 471105.511])],
 [array([257918.444, 471115.511])],
 [array([257918.444, 471125.511])],
 [array([257918.444, 471135.511])],
 [array([257918.444, 471145.511])],
 [array([257918.444, 471155.511])],
 [array([257918.444, 471165.511])],
 [array([257918.444, 471175.511])],
 [array([257918.444, 471185.511])],
 [array([257918.444, 471195.511])],
 [array([257918.444, 471205.511])],
 [array([257918.444, 471215.511])],
 [array([257918.444, 471225.511])],
 [array([257918.444, 471235.511])],
 [array([257918.444, 471245.511])],
 [array([257918.444, 471255.511])],
 [array([257918.444, 471265.511])],
 [array([257918.444, 471275.511])],
 [array([257918.444, 471285.511])],
 [array([257918.444, 471295.511])],
 [array([257918.444, 471305.511])],
 [array([257918.444, 471315.511])],
 [array([257918.444, 471325.511])],
 [array([257918.444, 471335.511])],
 [array([257918.444, 471345.

In [65]:
#points=np.stack(pixelsR2, axis=0)
points=np.array(pixelsR2)
points.shape
points=points.reshape(points.shape[0],points.shape[2])
points


array([[257918.444, 471075.511],
       [257918.444, 471085.511],
       [257918.444, 471095.511],
       ...,
       [258278.444, 471455.511],
       [258278.444, 471465.511],
       [258278.444, 471475.511]])

In [68]:
points3D=np.c_[points,np.zeros(points.shape[0])]
points3D

array([[257918.444, 471075.511,      0.   ],
       [257918.444, 471085.511,      0.   ],
       [257918.444, 471095.511,      0.   ],
       ...,
       [258278.444, 471455.511,      0.   ],
       [258278.444, 471465.511,      0.   ],
       [258278.444, 471475.511,      0.   ]])

In [70]:
geom = o3d.geometry.PointCloud()
geom.points = o3d.utility.Vector3dVector(points3D)
o3d.visualization.draw_geometries([geom])